grasp

In [ ]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

In [ ]:

gs.init(backend=gs.gpu, seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

In [6]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
# robot_path = "../assets/urdf/franka_description/robots/frankaEmikaPandaVirtual.urdf"
# franka_virtual = scene.add_entity(
#     gs.morphs.URDF(file=robot_path, fixed=True),
# )

franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda.xml",)
                   
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = [(0.7, 0, 0.015), (0.7, 2.0, 0.015), (0.7, -2.0, 0.015)]
cube, cube2, cube3 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
    ),
    surface=gs.surfaces.Default(
        color=(0.8, 0.1, 0.1),

    )
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


[Genesis] [13:54:58] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <b3678ba>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [13:54:58] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <7e1380c>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [13:54:59] [WARNING] (MJCF) Tendon not supported
[Genesis] [13:54:59] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [13:54:59] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [13:54:59] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [13:54:59] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [13:54:59] [INFO] Appl

In [7]:
n_envs = 0
scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

[Genesis] [13:55:02] [INFO] Building scene <ee8ae61>...
[Genesis] [13:55:04] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [13:55:04] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/13/25 13:55:05.825 9294] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [13:55:12] [INFO] Compiling simulation kernels...
[Genesis] [13:58:21] [INFO] Building visualizer...
[Genesis] [13:58:24] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [45]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
franka.control_dofs_position(franka.ready_qpos)
franka_MJCF.set_qpos(franka.ready_qpos)
franka_MJCF.control_dofs_position(franka.ready_qpos)

end_effector = franka.get_link('hand')
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())


scene.step()

[Genesis] [14:54:43] [INFO] Resetting Scene <ee8ae61>.
[Genesis] [14:54:43] [INFO] Running at 0.11 FPS.


In [37]:
cube_dq = adapter.get_dq(cube)
bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(bbox=bbox)

In [48]:
q = qs[1] 
s_axis = s_axes[1]
q, s_axis

(array([ 0.75    ,  0.      , -0.000981], dtype=float32),
 array([0., 1., 0.], dtype=float32))

In [49]:
cube.get_qpos()

tensor([0.7000, 0.0000, 0.0140, 1.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')

In [57]:
traj_pq = planner.generate_screw_trajectory(initial_pose=cube.get_qpos(),
                                  q=qs[1],
                                  s_axis=s_axes[1],
                                  output_type='p')

[[1.         0.         0.         0.69999999]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.014019  ]
 [0.         0.         0.         1.        ]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00  7.65000000e-01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16  4.90190117e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [62]:
adapter.get_size(cube)

array([0.10000923, 0.07      , 0.02999992])

In [43]:
traj_pq[-1]

array([0.750981  , 0.        , 0.749019  , 0.70710678, 0.        ,
       0.70710678, 0.        ])

In [58]:
for pq in traj_pq:
    cube.set_qpos(pq)
    scene.step()

[Genesis] [15:05:12] [INFO] Running at 0.02 FPS.
[Genesis] [15:05:12] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.03 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.04 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.04 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.04 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.04 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.04 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.05 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.05 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.05 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.05 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.06 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.06 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.06 FPS.
[Genesis] [15:05:13] [INFO] Running at 0.07 FPS.
[Genesis] [15:05:13]

In [ ]:
# target_quat = np.array([0, 1, 0, 0])
# center = np.array([0.5, 0, 0.5])
# r = 0.1
# damping = 1e-6
# diag = damping * np.eye(6)
# target_pos = center + np.array([1, 0, 0]) * r
# init_q = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = target_pos,
#     quat = target_quat,
# )
# franka.set_qpos(init_q)
# franka.control_dofs_position(init_q)
# for i in range(100):
#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
#     scene.step()

# for i in range(0, 500):
#     target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

#     # Position error.
#     error_pos = target_pos - end_effector.get_pos().cpu().numpy()

#     # Orientation error.
#     ee_quat = end_effector.get_quat().cpu().numpy()
#     error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
#     error_rotvec = gs.quat_to_rotvec(error_quat)

#     error = np.concatenate([error_pos, error_rotvec])

#     # jacobian
#     jac = franka.get_jacobian(link=end_effector).cpu().numpy()
#     dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
#     q = franka.get_qpos().cpu().numpy() + dq

#     # control
#     franka.control_dofs_position(q)
#     scene.step()


In [ ]:

# get the end-effector link
end_effector = franka.get_link('hand')
end_effector_MJCF = franka_MJCF.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
qpos_MJCF[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

In [ ]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


In [ ]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
path = franka.plan_path(
    qpos_goal     = qpos_reach,
    num_waypoints = 100, # 1s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_reach_MJCF,
    num_waypoints = 100, # 1s duration
)

for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

In [ ]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-5, -5]), fingers_dof)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[:-2], motors_dof)
franka_MJCF.control_dofs_force(np.array([-5, -5]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


In [ ]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

In [ ]:
adapter.step_simulation(5)

In [ ]:
ff = 0
franka.control_dofs_force(np.array([ff, ff]), fingers_dof)
franka_MJCF.control_dofs_force(np.array([ff, ff]), fingers_dof)
for _ in range(500):
    scene.step()